<a href="https://colab.research.google.com/github/rahiakela/natural-language-processing-research-and-practice/blob/main/nlp-for-semantic-search/2_sentence_transformers_basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Sentence Transformers Basics

**Reference**:

https://www.pinecone.io/learn/sentence-embeddings/#other-sentence-transformers


##Setup

In [ ]:
!pip -q install sentence-transformers

In [10]:
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import cos_sim

import numpy as np
import pandas as pd

##Getting Started with Sentence Transformers

We will start with the original SBERT model `bert-base-nli-mean-tokens`. 

First, we download and initialize the model.

In [ ]:
model = SentenceTransformer("bert-base-nli-mean-tokens")

In [4]:
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)

The output we can see here is the `SentenceTransformer` object which contains three components:

- The `transformer` itself, here we can see the max sequence length of 128 tokens and whether to lowercase any input (in this case, the model does not). We can also see the model class, `BertModel`.
- The `pooling` operation, here we can see that we are producing a 768-dimensional sentence embedding. We are doing this using the mean pooling method.

Once we have the model, building sentence embeddings is quickly done using the `encode` method.

In [5]:
sentences = [
   "the fifty mannequin heads floating in the pool kind of freaked them out",
    "she swore she just saw her sushi move",
    "he embraced his new life as an eggplant",
    "my dentist tells me that chewing bricks is very bad for your teeth",
    "the dental specialist recommended an immediate stop to flossing with construction materials"          
]

In [7]:
embeddings = model.encode(sentences)
embeddings.shape

(5, 768)

We can put together a fast STS example using nothing more than a cosine similarity function and Numpy.

In [9]:
sim = np.zeros((len(sentences), len(sentences)))

for i in range(len(sentences)):
  sim[i:, i] = cos_sim(embeddings[i], embeddings[i:])

sim

array([[1.00000048, 0.        , 0.        , 0.        , 0.        ],
       [0.40914333, 1.00000012, 0.        , 0.        , 0.        ],
       [0.1090903 , 0.44547975, 0.99999994, 0.        , 0.        ],
       [0.50074869, 0.30693942, 0.20791662, 0.9999997 , 0.        ],
       [0.29936224, 0.38607225, 0.28499281, 0.63849497, 1.00000024]])

<img src='https://d33wubrfki0l68.cloudfront.net/0c2178d0eadcc9c5755526afda3322b0efbf0b87/8059a/images/sentence-embeddings-9.jpg?raw=1' width='600'/>

Here we have calculated the cosine similarity between every combination of our five sentence embeddings. Which are:

In [15]:
# Creating pandas dataframe from numpy array
df = pd.DataFrame({"Sentence": sentences[:]})
df

,Sentence
0,the fifty mannequin heads floating in the pool...
1,she swore she just saw her sushi move
2,he embraced his new life as an eggplant
3,my dentist tells me that chewing bricks is ver...
4,the dental specialist recommended an immediate...


We can see the highest similarity score in the bottom-right corner with 0.64. As we would hope, this is for sentences 4 and 3, which both describe poor dental practices using construction materials.

##Other sentence-transformers

These newer models can significantly outperform the original SBERT.

In [16]:
models = ["all-mpnet-base-v2", "all-roberta-large-v1", "all-MiniLM-L12-v1"]
performances = [63.30, 53.05, 59.80]
speed = [2800, 800, 7500]
sizes = [418, 1355, 118]
df = pd.DataFrame({"Model": models[:], "Avg. Performance": performances[:], "Speed": speed[:], "Size(MB)": sizes[:]})
df

,Model,Avg. Performance,Speed,Size(MB)
0,all-mpnet-base-v2,63.30,2800,418
1,all-roberta-large-v1,53.05,800,1355
2,all-MiniLM-L12-v1,59.80,7500,118


For now, let’s compare one of the highest performers and run through our STS task.

In [ ]:
mpnet = SentenceTransformer("all-mpnet-base-v2")

In [18]:
mpnet

SentenceTransformer(
  (0): Transformer({'max_seq_length': 384, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
)

Let’s compare the STS results of `all-mpnet-base-v2` against SBERT.

In [19]:
embeddings = mpnet.encode(sentences)

sim = np.zeros((len(sentences), len(sentences)))

for i in range(len(sentences)):
  sim[i:, i] = cos_sim(embeddings[i], embeddings[i:])

sim

array([[ 1.00000024,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.26406288,  1.00000024,  0.        ,  0.        ,  0.        ],
       [ 0.16503499,  0.16126692,  1.00000024,  0.        ,  0.        ],
       [ 0.04334465,  0.04615876,  0.05670136,  1.00000048,  0.        ],
       [ 0.05398512,  0.06101187, -0.0112226 ,  0.51847208,  0.99999988]])

<img src='https://d33wubrfki0l68.cloudfront.net/988cdd0f93df1ffebf12f96621ffdabf861684b1/f8cb5/images/sentence-embeddings-10.jpg?raw=1' width='600'/>

The semantic representation of later models is apparent. Although SBERT correctly identifies 4 and 3 as the most similar pair, it also assigns reasonably high similarity to other sentence pairs.

On the other hand, the MPNet model makes a very clear distinction between similar and dissimilar pairs, with most pairs scoring less than 0.1 and the 4-3 pair scored at 0.52.